In [10]:
configPath = "metric_learning/config/cirtorch.yaml"
modelPath = "../../../Models/CIBR/rSfM120k-tl-resnet152-gem-w-f39cada.pth"
imageDatabasePath = "../../../Datasets/coco/images/train_corrected"
queryImagePath = "../../../Datasets/custom/Query.png"
folderToStoreResultsPath = "../../../Datasets/custom/COCO_Cirtorch/"

In [11]:
import os
import pickle
from metric_learning.networks import CirTorch, CirTorchConfig

os.environ['CUDA_VISIBLE_DEVICES'] = "0"

config = CirTorchConfig.create(configPath)
config.defrost()
config.model = modelPath
config.freeze()
cirtorchNetwork = CirTorch(config)
cirtorchNetwork.load()
databasePath = os.path.join(folderToStoreResultsPath, "database.pkl")
if os.path.exists(databasePath):
    cirtorchNetwork.loadDatabase(databasePath)
else:
    databaseImages = [os.path.join(imageDatabasePath, file) for file in os.listdir(imageDatabasePath)]
    cirtorchNetwork.createDatabase(databaseImages)
    cirtorchNetwork.saveDatabase(databasePath)
scores = cirtorchNetwork.query(queryImagePath)

scoresPath = os.path.join(folderToStoreResultsPath, "scores.pkl")
with open(scoresPath, 'wb') as scoresFile:
    pickle.dump(scores, scoresFile)


>> Loading network
>>>> 1/1 done...


In [13]:
import shutil

scoresPath = os.path.join(folderToStoreResultsPath, "scores.pkl")
with open(scoresPath, 'rb') as scoresFile:
    scores = pickle.load(scoresFile)
prevScore = 100
countSameScore = 0
for score in scores:
    if score[1] > 0.10:
        scoreAsInteger = int(score[1]*100)
        if prevScore != scoreAsInteger:
            countSameScore = 1
        else:
            countSameScore += 1
        prevScore = scoreAsInteger
        imagePath = cirtorchNetwork.images[int(score[0])]
        shutil.copy(imagePath, os.path.join(folderToStoreResultsPath, f"{scoreAsInteger}_{countSameScore}_{os.path.split(imagePath)[-1]}"))

KeyboardInterrupt: 